# Import libraries 

In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import random

# Load the Data

In [ ]:
# Ecxel file path
faq_file_path = r"D:\Ammar\Chatbot for FAQs\FAQ_data.xlsx"

# Read FAQ data
def load_faq_data(file_path):
    try:
        df = pd.read_excel(file_path)
        #  Make sure the "Question" and "Answer" columns are present
        if 'Question' in df.columns and 'Answer' in df.columns:
            print("Data loaded successfully.")  # التحقق من تحميل البيانات
            return dict(zip(df['Question'], df['Answer']))
        else:
            print("Error: 'Question' or 'Answer' columns are missing in the Excel file.")
            return {}
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return {}

# Load FAQ Data
faq_data = load_faq_data(faq_file_path)

# Chatbot

In [ ]:
# function to find the answer from the dataset
def find_answer(question):
    question = question.strip().lower()  
    if not faq_data:
        return "Sorry, no data loaded from the file."
    
    #Flexible data search (try to find the question partially or match it precisely)
    for q in faq_data:
        if question in q.lower():  # Partial match
            print(f"Match found: {q}")  # print the question
            answer = faq_data[q]
            print(f"Answer found: {answer}")  # print the question the answer
            return answer
    return "Sorry, no answer found for this question."

# GUI Update
def search():
    question = question_entry.get()
    if question.strip() == "":  # if the question empty
        answer_label.config(foreground="red", text="Please enter a question.")
    else:
        answer = find_answer(question)
        answer_label.config(foreground="black", text=answer)  
        app.update()  # Complete GUI update to ensure answer display.

# function to select seggested questions
def select_suggested_question(question):
    question_entry.delete(0, tk.END)
    question_entry.insert(0, question)
    search()  # بعد تحديد السؤال المقترح، نبحث مباشرة

# suggested_questions update
def update_suggested_questions():
    if not faq_data:
        return  
    
    # Extracting 5 random questions
    suggested_questions = random.sample(list(faq_data.keys()), 5)  # الحصول على 5 أسئلة عشوائية

    # Cleaning the framework of proposed questions before adding new questions.
    for widget in suggested_questions_frame.winfo_children():
        widget.destroy()

    # Adding new questions to the GUI
    for question in suggested_questions:
        btn = ttk.Button(
            suggested_questions_frame, 
            text=question, 
            command=lambda q=question: select_suggested_question(q),
            width=50
        )
        btn.pack(pady=2)

# `GUI`

In [ ]:
# GUI 
app = tk.Tk()
app.title("Chatbot FAQ")
app.geometry("600x500")
app.resizable(False, False)

# Enter Question
question_label = ttk.Label(app, text="Enter your question:", font=("Arial", 12))
question_label.pack(pady=10)

question_entry = ttk.Entry(app, width=50, font=("Arial", 12))
question_entry.pack(pady=5)

# Search button
search_button = ttk.Button(app, text="Search", command=search)
search_button.pack(pady=20)

# suggeste questions
suggested_questions_label = ttk.Label(app, text="Suggested Questions:", font=("Arial", 12))
suggested_questions_label.pack(pady=10)

suggested_questions_frame = tk.Frame(app)
suggested_questions_frame.pack()

# Refresh button
refresh_button = ttk.Button(app, text="Refresh Questions", command=update_suggested_questions)
refresh_button.pack(pady=10)

# Show Answr
answer_label = ttk.Label(app, text="", wraplength=500, justify="center", font=("Arial", 12))
answer_label.pack(pady=20)

# Run the APP
update_suggested_questions()   
app.mainloop()


# `Flask_APP`

In [ ]:
from flask import Flask, render_template_string, request, jsonify, session
import pandas as pd

# Load data
df = pd.read_excel(r"D:\Ammar\Chatbot for FAQs\FAQ_data.xlsx") 

# make a dict for questions and answers
questions_answers = dict(zip(df['Question'], df['Answer']))  

app = Flask(__name__)
app.secret_key = 'your_secret_key' #Secret key to start the session

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Ask for Username 
        username = request.form.get('username')
        session['username'] = username
        return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Customer Service Chat</title>
            <style>
                body {
                    font-family: Arial, sans-serif;
                    background-color: #f4f7fa;
                    margin: 0;
                    padding: 0;
                }
                .container {
                    width: 100%;
                    max-width: 800px;
                    margin: 0 auto;
                    padding: 20px;
                    background-color: #fff;
                    border-radius: 10px;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                }
                .chat-box {
                    height: 400px;
                    overflow-y: scroll;
                    margin-bottom: 20px;
                    border: 1px solid #ddd;
                    padding: 10px;
                    border-radius: 10px;
                    background-color: #f9f9f9;
                }
                .user-message {
                    text-align: right;
                    margin: 10px 0;
                    font-size: 14px;
                    background-color: #e1f5fe;
                    padding: 8px;
                    border-radius: 10px;
                }
                .bot-message {
                    text-align: left;
                    margin: 10px 0;
                    font-size: 14px;
                    background-color: #dcedc8;
                    padding: 8px;
                    border-radius: 10px;
                }
                input[type="text"] {
                    width: calc(100% - 90px);
                    padding: 10px;
                    border-radius: 5px;
                    border: 1px solid #ccc;
                }
                button {
                    padding: 10px 20px;
                    border: none;
                    background-color: #007bff;
                    color: #fff;
                    border-radius: 5px;
                    cursor: pointer;
                }
                button:hover {
                    background-color: #0056b3;
                }
                .faq-button {
                    margin-top: 10px;
                    background-color: #28a745;
                }
                .faq-button:hover {
                    background-color: #218838;
                }
                .new-chat-button {
                    background-color: #dc3545;
                }
                .new-chat-button:hover {
                    background-color: #c82333;
                }
                .question-button {
                    margin-top: 10px;
                    background-color: #17a2b8;
                    margin-right: 5px;
                    padding: 5px;
                    border: none;
                    color: white;
                    border-radius: 5px;
                    cursor: pointer;
                }
                .question-button:hover {
                    background-color: #138496;
                }
            </style>
        </head>
        <body>

            <div class="container">
                <h2>Customer Service Chat</h2>
                <p id="greeting"></p>
                <div class="chat-box" id="chat-box"></div>
                <input type="text" id="user-input" placeholder="Ask a question..." onkeyup="showSuggestions()" />
                <button onclick="sendMessage()">Send</button>
                <button class="faq-button" onclick="refreshQuestions()">FAQ</button>
                <button class="new-chat-button" onclick="newChat()">New Chat</button>
                <div id="suggestions"></div>  <!suggested questions>
            </div>

            <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
            <script>
                const chatBox = document.getElementById('chat-box');
                const userInput = document.getElementById('user-input');
                const suggestions = document.getElementById('suggestions');
                const greeting = document.getElementById('greeting');

                // Display a personalized greeting when the page loads.
                window.onload = function() {
                    const username = "{{ session['username'] }}";
                    if (username) {
                        greeting.textContent = `Hello, ${username}! How can I assist you today?`;
                    } else {
                        greeting.textContent = "Hello! How can I assist you today?";
                    }
                    suggestions.innerHTML = ""; // Hide suggestions at the beginning.
                }

                // send massege
                function sendMessage() {
                    const question = userInput.value;
                    if (question.trim() === "") return;

                    appendMessage(question, 'user');
                    userInput.value = "";  

                    // Send the question to Flask
                    $.post("/ask", { question: question }, function(data) {
                        appendMessage(data.answer, 'bot');
                    });
                }

                // Add a message to the window
                function appendMessage(message, sender) {
                    const messageDiv = document.createElement('div');
                    messageDiv.classList.add(sender + '-message');
                    messageDiv.textContent = message;
                    chatBox.appendChild(messageDiv);
                    chatBox.scrollTop = chatBox.scrollHeight;
                }

                // Fetching proposed questions
                function refreshQuestions() {
                    $.get("/suggest_questions", function(data) {
                        suggestions.innerHTML = "";
                        data.questions.forEach(function(question) {
                            const button = document.createElement('button');
                            button.textContent = question;
                            button.classList.add("question-button");
                            button.onclick = function() {
                                userInput.value = question;
                                sendMessage();
                            };
                            suggestions.appendChild(button);
                        });
                    });
                }

                // Display suggestions while typing
                function showSuggestions() {
                    const input = userInput.value.toLowerCase();
                    suggestions.innerHTML = "";  // Delete old suggestions

                    if (input.trim() === "") return;  

                    // Filtering questions based on the input text
                    $.get("/filter_questions", { input: input }, function(data) {
                        data.questions.forEach(function(question) {
                            const button = document.createElement('button');
                            button.textContent = question;
                            button.classList.add("question-button");
                            button.onclick = function() {
                                userInput.value = question;
                                sendMessage();
                            };
                            suggestions.appendChild(button);
                        });
                    });
                }

                // Start new chat
                function newChat() {
                    chatBox.innerHTML = '';
                    userInput.value = '';
                    suggestions.innerHTML = '';
                    greeting.textContent = "Hello! How can I assist you today?";
                }
            </script>

        </body>
        </html>
        ''')

    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Customer Service Chat</title>
    </head>
    <body>
        <h2>Enter your name to start the chat</h2>
        <form method="POST">
            <label for="username">Your Name: </label>
            <input type="text" name="username" id="username" required />
            <button type="submit">Start Chat</button>
        </form>
    </body>
    </html>
    ''')

@app.route('/ask', methods=['POST'])
def ask():
    user_question = request.form['question']
    answer = questions_answers.get(user_question, "Sorry, I don't have an answer for that.")
    return jsonify({'answer': answer})

@app.route('/suggest_questions', methods=['GET'])
def suggest_questions():
    sample_questions = df['Question'].sample(n=5).tolist()  # 5 random questions
    return jsonify({'questions': sample_questions})

@app.route('/filter_questions', methods=['GET'])
def filter_questions():
    input_text = request.args.get('input', '').lower()
    filtered_questions = df['Question'][df['Question'].str.lower().str.contains(input_text)].tolist()
    return jsonify({'questions': filtered_questions})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)  


# Summary 

The project is a chatbot application designed to provide answers to frequently asked questions (FAQs) using data sourced from an Excel file. It consists of two main components: a desktop application built with Tkinter and a web application using Flask.

### Key Features:

1. **Data Loading**: The application reads FAQ data from an Excel file, ensuring that the necessary "Question" and "Answer" columns are present. It creates a dictionary mapping questions to their corresponding answers.

2. **Chatbot Functionality**:
   - The chatbot can find answers based on user input, allowing for both exact and partial matches of questions.
   - Users can receive suggested questions, which are randomly selected from the dataset, enhancing the user experience by providing options.

3. **Graphical User Interface (GUI)**: 
   - The Tkinter application features an entry field for user questions, a search button, and a display area for answers. It also includes a refresh button to update suggested questions.
   - The GUI is designed to be user-friendly, with clear labels and a structured layout.

4. **Web Application**: 
   - The Flask application serves a web interface where users can enter their names to start a chat session.
   - It handles user questions via AJAX requests, providing answers and suggested questions dynamically without needing to refresh the page.
   - The web interface includes a chat box for displaying messages, input fields for user questions, and buttons for sending messages and refreshing suggestions.

5. **Session Management**: The Flask app uses session management to personalize the chat experience, greeting users by name.

6. **Dynamic Suggestions**: The application can filter and suggest questions based on user input, enhancing the interactivity and responsiveness of the chatbot.

Overall, the project combines a robust backend for data handling with an intuitive frontend for user interaction, making it a comprehensive solution for addressing FAQs through a chatbot interface.